# Solution to live session: Modelling Onchocerciasis

The following code simulates the basic onchocerciasis model for the annual biting rate (ABR) in village 1 (Tcholliré) and plots the predicted outputs. Running the code for different values of ABR in different villages allows you to complete the table in the practical.
  
[1. Run code](#code)  
[2. Plot outputs](#plots)
[2.1 Plot outputs over time: base R](#baseplot)  
[2.2 Plot outputs over time: ggplot](#ggplot)  
[4. Extract the outputs to fill in the table](#outputs)  
[5. Plot observed and predicted values against ABR](#obspredABR)  
[6. Structural uncertainty: quadratic PIVM](#quadPIVM)
<a name = "top"></a>

## 1. Fill in the parameters and run the model code <a name = "code"></a>

In [ ]:
# PACKAGES
require(deSolve)
require(reshape2)
require(ggplot2)


In [ ]:
# INPUT

# Initial conditions
initial_state_values <- c(W = 10,  # mean number of adult worms per person at time 0
                          M = 0,   # mean number of Mf per milligram per person at time 0
                          L = 0,   # mean number of infective larvae (L3) per fly at time 0
                          P = 0)   # microfilarial prevalence output

# Parameter values (values are expressed per year)
# Average values taken from Table 2 Basáñez & Boussinesq (1999)
aH <- 0.8
sH <- 0.2       
aV <- 0.4481    
sV0 <- 0.0463   
c <- 0.0196
alpha <- 0.8653
h <- 0.3         
g <- 0.0096

parameters <- c(aH = aH,         # the proportion of larvae shed per bite
                sH = sH,         # the fraction of inoculated L3 larvae surviving and reaching maturity within the human host
                aV = aV,         # the fraction of Mf ingested per bite per milligram of skin
                sV0 = sV0,       # the fraction of ingested Mf reaching infectivity within the fly
                c = c,
                alpha = alpha,
                h = h,           # the human-blood index (fraction of blood-meals taken on humans)
                g = g,           # length of the gonotrophic cycle
                deltaH0 = aH*sH, # the proportion of L3 larvae developing to the adult stage within the human host, 
                                 # per bite, when the transmission potential tends to 0
                deltaHinfinity = 0.0032,  # the proportion of L3 larvae developing to the adult stage within the human 
                                          # host, per bite, when the transmission potential tends to infinity
                cH = 0.0137,     # the reciprocal of the ATP value for which deltaH(ATP) = (deltaH0+deltaHinfinity)/2
                muH = 0.02,      # per capita death rate of human host
                sigmaW = 0.1,    # per capita death rate of female adult worms
                sigmaM = 0.8,    # per capita death rate of dermal Mf
                sigmaL = 104,    # per capita death rate of L3 larvae within the fly host
                phi = 1,         # mating probability (polygamous, dioecious worms)
                F = 0.6674,      # per capita fecundity rate of adult female worms scaled by the total weight 
                                 # of Mf-bearing skin
                k0 = 0.0553,     # fitted parameter for microfilarial prevalence as a function of mean microfilarial load per person
                k1 = 0.491,      # fitted parameter for microfilarial prevalence as a function of mean microfilarial load per person
                deltaV0 = aV*sV0,      # the proportion of Mf mg-1 developing to the infective stage within unarmed 
                                       # vectors, per Mf, per bite, when M tends to 0
                cV0 = aV*c,            # the severity of density-dependent limitation of 
                                       # larval development within unarmed vectors, per dermal Mf
                alphaV = aV*alpha,     # the per capita excess vector mortality induced by the Mf parasite stage
                beta = h/g,            # the biting rate per fly on humans
                muV = 52,              # per capita death rate of vector host (uninfected)
                ABR = 1000)            # village-specific annual biting rate

# Simulate model for 300 years, in 0.1 year intervals
timestep <- 0.1
times <- seq(from = 0, to = 300, by = timestep)

# ONCHO MODEL FUNCTION
oncho_model <- function(time, state, parameters) {  
  
  with(as.list(c(state, parameters)), {
    
    # Differential equations
    dW <-((deltaH0 + deltaHinfinity * cH * ABR * L)/(1 + cH * ABR * L)) * ABR * L - (sigmaW + muH) * W
    dM <- ((1/2) * phi * F) * W - (sigmaM + muH) * M
    dL <- (deltaV0/(1 + cV0 * M)) * beta * M - (sigmaL + muV + alphaV * M + (aH/g)) * L

    # Microfilarial prevalence (in %)
    P <- 100 * (1 - (1 + M/(k0*M^k1))^(-k0*M^k1))
    
    # Output
    return(list(c(dW, dM, dL, P))) 
  })
}

# Solve the equations
output <- as.data.frame(ode(y = initial_state_values, 
                            times = times, 
                            func = oncho_model,
                            parms = parameters))


# The microfilarial prevalence is returned as a cumulative value when solving the model, so we need to 
# convert this into a prevalence measure by substracting the cumulative incidence at each timestep t by the cumulative
# incidence at time t-1
output$microfilarial_prevalence <- c(output$P[1], diff(output$P/timestep, lag = 1))  


If you run the above code with each of the different ABRs that are given in the reference paper, then you will be able to fill in the table.

## 2. Plot the outputs over time using base R <a name = "baseplot"></a> ([return to top](#top))

In [ ]:
## plots: base R

# Plot the mean number in each compartment over time 

par(mfrow = c(1,3))

# Plot the microfilarial prevalence over time
plot(output$time, output$microfilarial_prevalence, type = 'l',                                                       
     xlab = "Time (years)", ylab = "Mf prevalence")  

# Plot the adult worm intensity over time
plot(output$time, output$W, type = 'l',                                                       
     xlab = "Time (years)", ylab = "Mean number", ylim = c(0, 25))  

# Plot the microfilarial intensity over time
lines(output$time, output$M, col = 'red')

legend(80, 5, legend=c("adult worms", "mf"),
       lty=c(1, 1), col = c('black', 'red'),  cex=0.8)

plot(output$time, output$L, xlab = "Time (years)", ylab = "Mean L3", type = 'l')



In [ ]:
## Plots: ggplot

output_long <- melt(as.data.frame(output), id = "time")                  # turn output dataset into long format

# Calculating the predicted annual transmission potential (ATP) for a given village:
paste("ATP = ", parameters["ABR"] * output$L[output$time==300])

# Plot the mean number in each compartment over time 
ggplot(data = output_long[output_long$variable != "P",],                                               
       aes(x = time, y = value, colour = variable, group = variable)) +  
  geom_line() +                                                          
  xlab("Time (years)")+                                                   
  ylab("Mean number") +                                     
  labs(colour = "Compartment")

# Plot the microfilarial prevalence over time
ggplot(data = output,                                               
       aes(x = time, y = microfilarial_prevalence)) +  
  geom_line() +                                                          
  xlab("Time (years)")+                                                   
  ylab("Microfilarial prevalence (%)")


In [ ]:
# Calculating the predicted annual transmission potential (ATP) for a given village:
paste("ATP = ", parameters["ABR"] * output$L[output$time==300])

# Extract the required predictions from the model: the equilibrium levels of P, M, L
output[(nrow(output)), ] # this line of code will select the last row of the dataframe

# when you add these to the table, make note of the order they are in!


## Structural assumptions on parasite-induced vector mortality: quadratic relationship

In [ ]:
# Adding parameter values:
alpha1 <- 0.22
parameters["alphaV1"] <- aV^2*alpha1

# Modify the equation for the infective larvae per fly, dL:

oncho_model <- function(time, state, parameters) {  
  
  with(as.list(c(state, parameters)), {
    
    # Differential equations
    dW <-((deltaH0 + deltaHinfinity * cH * ABR * L)/(1 + cH * ABR * L)) * ABR * L - (sigmaW + muH) * W
    dM <- ((1/2) * phi * F) * W - (sigmaM + muH) * M
    dL <- (deltaV0/(1 + cV0 * M)) * beta * M - (sigmaL + muV + alphaV1 * M^2 + (aH/g)) * L

    # Microfilarial prevalence (in %)
    P <- 100 * (1 - (1 + M/(k0*M^k1))^(-k0*M^k1))
    
    # Output
    return(list(c(dW, dM, dL, P))) 
  })
}

# Solve the equations
output <- as.data.frame(ode(y = initial_state_values, 
                            times = times, 
                            func = oncho_model,
                            parms = parameters))

output_long <- melt(as.data.frame(output), id = "time")                  # turn output dataset into long format

# The microfilarial prevalence is returned as a cumulative value when solving the model, so we need to 
# convert this into a prevalence measure by substracting the cumulative incidence at each timestep t by the cumulative
# incidence at time t-1
output$microfilarial_prevalence <- c(output$P[1], diff(output$P/timestep, lag = 1))  

# Calculating the predicted annual transmission potential (ATP) for a given village:
paste("ATP = ", parameters["ABR"] * output$L[output$time==300])

# Plot the mean number in each compartment over time 
ggplot(data = output_long[output_long$variable != "P",],                                               
       aes(x = time, y = value, colour = variable, group = variable)) +  
  geom_line() +                                                          
  xlab("Time (years)")+                                                   
  ylab("Mean number") +                                     
  labs(colour = "Compartment")   

# Plot the microfilarial prevalence over time
ggplot(data = output,                                               
       aes(x = time, y = microfilarial_prevalence)) +  
  geom_line() +                                                          
  xlab("Time (years)")+                                                   
  ylab("Microfilarial prevalence (%)")